In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras import Input
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import *

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((X_train.shape[0], 28 * 28 * 1))
X_test = X_test.reshape((X_test.shape[0], 28 * 28 * 1))

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8)

In [3]:
sequential_model = Sequential()
sequential_model.add(Dense(256, input_shape=(28 * 28 * 1,), activation='sigmoid'))
sequential_model.add(Dense(128, activation='sigmoid'))
sequential_model.add(Dense(10, activation='softmax'))

In [4]:
layers = [Dense(256, input_shape=(28 * 28 * 1,), activation='sigmoid'),
          Dense(128, activation='sigmoid'),
          Dense(10, activation='softmax')]
sequential_model2 = Sequential(layers)

In [5]:
input_layer = Input(shape=(28 * 28 * 1,))
dense_1 = Dense(256, activation='sigmoid')(input_layer)
dense_2 = Dense(128, activation='sigmoid')(dense_1)
predictions = Dense(10, activation='softmax')(dense_2)
functional_model = Model(inputs=input_layer, outputs=predictions)

In [6]:
class ClassModel(Model):
    def __init__(self):
        super(ClassModel, self).__init__()

        self.dense_1 = Dense(256, activation='sigmoid')
        self.dense_2 = Dense(256, activation='sigmoid')
        self.predictions = Dense(10, activation='softmax')

    def call(self, inputs, **kwargs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)

        return self.predictions(x)

class_model = ClassModel()

In [7]:
models = {
    'sequential_model': sequential_model,
    'sequential_model_list': sequential_model2,
    'functional_model': functional_model,
    'class_model': class_model
}

In [8]:
for name, model in models.items():
    print(f'Compiling model: {name}')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    print(f'Training model: {name}')
    model.fit(X_train, y_train,
              validation_data=(X_valid, y_valid),
              epochs=50,
              batch_size=256,
              verbose=0)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Testing model: {name}. \nAccuracy: {accuracy}')
    print('---')

Compiling model: sequential_model
Training model: sequential_model
Testing model: sequential_model. 
Accuracy: 0.9797999858856201
---
Compiling model: sequential_model_list
Training model: sequential_model_list
Testing model: sequential_model_list. 
Accuracy: 0.9804999828338623
---
Compiling model: functional_model
Training model: functional_model
Testing model: functional_model. 
Accuracy: 0.9800999760627747
---
Compiling model: class_model
Training model: class_model
Testing model: class_model. 
Accuracy: 0.9811000227928162
---
